- In the CRSS dataset, do two things.  
    - Bin into fewer categories, then
    - Impute unknown values
    

In [1]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Setup
## Import Libraries

In [2]:
import sys, copy, math, time, os

print ('Python version: {}'.format(sys.version))

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)


import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)

# Library for reading Microsoft Access files
import pandas_access as mdb


# Set Randomness.  Copied from https://www.kaggle.com/code/abazdyrev/keras-nn-focal-loss-experiments
import random


Python version: 3.9.16 (main, Dec  7 2022, 10:02:13) 
[Clang 14.0.0 (clang-1400.0.29.202)]
NumPy version: 1.24.0
Pandas version:  1.5.2


# Import Data

### accident.csv from CRSS

In [3]:
def Import_Data_Accident(NAMES):
    print ('Import_Data_Accident()')

    df = pd.DataFrame([])
#    for year in ['2018']:
    for year in ['2016','2017','2018']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/ACCIDENT.CSV'
        temp = pd.read_csv(filename, index_col=None)
        print (year, len(temp))
        df = df.append(temp)

#    for year in ['2020']:
    for year in ['2019','2020']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/accident.csv'
        temp = pd.read_csv(filename, index_col=None)
        print (year, len(temp))
        df = df.append(temp)
    
    if NAMES==0:
        for feature in df:
            if 'NAME' in feature:
                df.drop(columns=[feature], inplace=True)

    print (df.shape)
    print ()
    return df

## vehicle.csv from CRSS

In [4]:
def Import_Data_Vehicle(NAMES):
    print ('Import_Data_Vehicle()')

    df = pd.DataFrame([])
    for year in ['2016','2017','2018']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/VEHICLE.CSV'
        temp = pd.read_csv(filename, index_col=None, low_memory=False)
        print (year, len(temp))
        df = df.append(temp)

    for year in ['2019','2020']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/vehicle.csv'
        temp = pd.read_csv(filename, index_col=None, encoding='latin1', low_memory=False)
        print (year, len(temp))
        df = df.append(temp)

    if NAMES==0:
        for feature in df:
            if 'NAME' in feature:
                df.drop(columns=[feature], inplace=True)

    print (df.shape)
    print ()
    return df

### person.csv from CRSS

In [5]:
def Import_Data_Person(NAMES):
    print ('Import_Data_Person()')

    df = pd.DataFrame([])
    for year in ['2016','2017','2018']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/PERSON.CSV'
        temp = pd.read_csv(filename, index_col=None)
        print (year, len(temp))
        df = df.append(temp)

    for year in ['2019','2020']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/person.csv'
        temp = pd.read_csv(filename, index_col=None, encoding='latin1')
        print (year, len(temp))
        df = df.append(temp)

    if NAMES==0:
        for feature in df:
            if 'NAME' in feature:
                df.drop(columns=[feature], inplace=True)

    print (df.shape)
    print ()
    return df

## Get Data
- The Get_Data_from_Original() reads the (original) CRSS files from the CRSS directory, preprocesses it, and writes it to files in a folder outside this GitHub repo (because the files are too large for my subscription), and returns the dataframes.
- The Get_Data_from_Temp_Files() reads the temp files and returns the dataframes.  I created this option for running repeatedly during writing and debugging, because it's much faster.

In [6]:
def Get_Data_from_Original():
    print ('Get_Data_from_Original()')
    
    df_Accident = Import_Data_Accident(0)
    df_Vehicle = Import_Data_Vehicle(0)
    df_Person = Import_Data_Person(0)
    
    df_Accident.to_csv('../../Big_Files/Accident.csv', index=False)
    df_Vehicle.to_csv('../../Big_Files/Vehicle.csv', index=False)
    df_Person.to_csv('../../Big_Files/Person.csv', index=False)
    

    df_Accident = Import_Data_Accident(1)
    df_Vehicle = Import_Data_Vehicle(1)
    df_Person = Import_Data_Person(1)
    
    df_Accident.to_csv('../../Big_Files/Accident_with_NAMES.csv', index=False)
    df_Vehicle.to_csv('../../Big_Files/Vehicle_with_NAMES.csv', index=False)
    df_Person.to_csv('../../Big_Files/Person.csv_with_NAMES', index=False)
    

    return df_Accident, df_Vehicle, df_Person

In [7]:
def Get_Data_from_Temp_Files():
    print ('Get_Data_from_Temp_File')
    df_Accident = pd.read_csv('../../Big_Files/Accident.csv')
    df_Vehicle = pd.read_csv('../../Big_Files/Vehicle.csv', low_memory=False)
    df_Person = pd.read_csv('../../Big_Files/Person.csv')
    print ()
    
    return df_Accident, df_Vehicle, df_Person    

In [8]:
#df_Accident, df_Vehicle, df_Person = Get_Data_from_Original()

In [9]:
df_Accident, df_Vehicle, df_Person = Get_Data_from_Temp_Files()

Get_Data_from_Temp_File



# Accident Dataset

- This information is in the Accident_Dataset_Information.xlsx file in the Keras folder.

|  Feature  |  Meaning  |  Number of Values  |  Number of Missing Values  | Continuous, Categorical, Count, Mixed, Transfer, or Drop | Values Signifying ''Missing'' | Number of Samples signified as "Missing" |  Notes  | 
 | --- | --- | --- | --- | --- | --- | --- | --- |
 | ALCHL_IM | ALCOHOL Imputed | 2 | 0 | Categorical |  |  |  | 
 | ALCOHOL | Alcohol Involved in Crash  | 4 | 0 | Categorical | [9] | 59889 | Derived Data Element | 
 | CASENUM |  | 259077 | 0 | Transfer |  |  |  | 
 | CF1 |  | 23 | 54745 | Drop |  |  | Discontinued | 
 | CF2 |  | 17 | 54745 | Drop |  |  | Discontinued | 
 | CF3 |  | 11 | 54745 | Drop |  |  | Discontinued | 
 | DAY_WEEK |  | 7 | 0 | Categorical | [9] | 0 |  | 
 | EVENT1_IM | HARM_EV Imputed | 54 | 0 | Categorical |  |  |  | 
 | HARM_EV | First Harmful Event  | 56 | 0 | Categorical | [98,99] | 166 |  | 
 | HOUR |  | 25 | 0 | Categorical | [99] | 1127 |  | 
 | HOUR_IM |  | 24 | 0 | Categorical |  |  |  | 
 | INT_HWY | Interstate Highway | 3 | 0 | Categorical | [9] | 25 |  | 
 | LGTCON_IM | LGT_COND Imputed | 7 | 0 | Categorical |  |  |  | 
 | LGT_COND | Light Condition | 9 | 0 | Categorical | [8,9] | 2309 |  | 
 | MANCOL_IM | MAN_COLL Imputed | 9 | 0 | Categorical |  |  |  | 
 | MAN_COLL | Manner of Collision of the First Harmful Event  | 11 | 0 | Categorical | [98,99] | 1012 |  | 
 | MAXSEV_IM | MAX_SEV Imputed | 8 | 0 | Categorical |  |  | Derived Data Element | 
 | MAX_SEV | Maximum Severity in Crash | 9 | 0 | Categorical | [9] | 4480 | Derived Data Element | 
 | MINUTE |  | 61 | 0 | Categorical | [99] | 1127 |  | 
 | MINUTE_IM |  | 60 | 0 | Categorical |  |  |  | 
 | MONTH |  | 12 | 0 | Categorical |  |  |  | 
 | NO_INJ_IM | NUM_INJ Imputed | 18 | 0 | Count |  |  | Derived Data Element | 
 | NUM_INJ | Number Injured in Crash | 20 | 0 | Count | [99] | 4480 | Change 98 to 0; derived data element | 
 | PEDS | Number of persons not in motor vehicles | 10 | 0 | Count |  |  |  | 
 | PERMVIT | Number of Persons in Motor Vehicles in Transport  | 26 | 0 | Count |  |  |  | 
 | PERNOTMVIT | Number of Persons Not in Motor Vehicles in Transport  | 10 | 0 | Count |  |  |  | 
 | PJ |  | 422 | 0 | Drop |  |  |  | 
 | PSU |  | 60 | 0 | Drop |  |  |  | 
 | PSUSTRAT |  | 25 | 0 | Drop |  |  |  | 
 | PSU_VAR |  | 67 | 0 | Drop |  |  |  | 
 | PVH_INVL | Number of Parked/Working Vehicles in the Crash  | 11 | 0 | Count |  |  |  | 
 | REGION |  | 4 | 0 | Drop |  |  |  | 
 | RELJCT1 | Relation to Junction-Within Interchange Area  | 4 | 0 | Categorical | [8,9] | 65920 |  | 
 | RELJCT1_IM | RELJCT1 Imputed | 3 | 54409 | Categorical |  |  |  | 
 | RELJCT2 | Relation to Junction-Specific Location  | 15 | 0 | Categorical | [98,99] | 19721 |  | 
 | RELJCT2_IM | RELJCT2 Imputed | 13 | 0 | Categorical |  |  |  | 
 | REL_ROAD | Relation to Trafficway  | 13 | 0 | Categorical | [98,99] | 190 |  | 
 | SCH_BUS |  | 2 | 0 | Categorical |  |  |  | 
 | STRATUM |  | 9 | 0 | Drop |  |  |  | 
 | TYP_INT | Type of Intersection  | 11 | 0 | Categorical | [98,99] | 26650 |  | 
 | URBANICITY |  | 2 | 0 | Categorical |  |  |  | 
 | VE_FORMS | Number of Motor Vehicles in Transport  | 13 | 0 | Count |  |  |  | 
 | VE_TOTAL | Number of vehicles in crash | 13 | 0 | Count |  |  |  | 
 | WEATHER |  | 13 | 0 | Categorical | [98,99] | 13284 |  | 
 | WEATHER1 |  | 14 | 54745 | Drop |  |  | Discontinued | 
 | WEATHER2 |  | 14 | 54745 | Drop |  |  | Discontinued | 
 | WEATHR_IM | WEATHER Imputed | 11 | 0 | Categorical |  |  |  | 
 | WEIGHT | Case weight | 8816 | 0 | Drop |  |  |  | 
 | WKDY_IM | DAY_WEEK Imputed | 7 | 0 | Categorical |  |  |  | 
 | WRK_ZONE | Work Zone | 5 | 0 | Categorical |  |  |  | 
 | YEAR |  | 5 | 0 | Categorical |  |  |  | ![image.png](attachment:image.png)

# Organize Data

In [10]:
# Fix Accident.RELJCT1_IM, which has all of 2019 missing.
# Make RELJCT1_IM2, which puts values from RELJCT1 into blank spots in RELJCT1_IM

def Fix_RELJCT1_IM(df_Accident):
    df_Accident['RELJCT1_IM2'] = df_Accident.RELJCT1_IM.fillna(df_Accident.RELJCT1)
    print ('RELJCT1.value_counts()')
    print (df_Accident.RELJCT1[df_Accident.YEAR==2019].value_counts())
    print ()
    print ('RELJCT1_IM.value_counts()')
    print (df_Accident.RELJCT1_IM[df_Accident.YEAR==2019].value_counts())
    print ()
    print ('RELJCT1_IM2.value_counts()')
    print (df_Accident.RELJCT1_IM2[df_Accident.YEAR==2019].value_counts())
    print ()
    print ('YEAR.value_counts()')
    print (df_Accident.YEAR.value_counts())
    return df_Accident
    
df_Accident = Fix_RELJCT1_IM(df_Accident)

RELJCT1.value_counts()
8    34235
0    18699
1     1449
9       26
Name: RELJCT1, dtype: int64

RELJCT1_IM.value_counts()
Series([], Name: RELJCT1_IM, dtype: int64)

RELJCT1_IM2.value_counts()
8.0    34235
0.0    18699
1.0     1449
9.0       26
Name: RELJCT1_IM2, dtype: int64

YEAR.value_counts()
2017    54969
2020    54745
2019    54409
2018    48443
2016    46511
Name: YEAR, dtype: int64


In [11]:
# Understand the missing values in TYP_INT
# Doesn't seem to be related to any other feature.  Just random.

for feature in df_Accident:
    if len(df_Accident[feature].unique())<20:
        print (pd.crosstab(df_Accident.TYP_INT, df_Accident[feature], normalize=False))
        print ()

STRATUM    2     3     4      5      6      7      8      9      10
TYP_INT                                                            
1        9334  8451  4102  10203  18024  10446  16660  32159  33200
2        6384  2693  1920   3227  11938   2994   8729  13744   9768
3        2756  1619   678   1390   4197   1523   3508   5773   4947
4          56    51    14     51    123     50    118    203    159
5          18    15     4      3     13     13     11     60     43
6          40    42     9     16     49     49     25    161    107
7          31    23    11     31     79     22     51     96     63
10         15    10     2      8     14     11     14     21     49
11          1     1     0      2      1      0      0      1      0
98       3456  1014   498    655   4316   1524   2813   7537   4752
99         21     1     1      7      7      3      4     21     20

VE_TOTAL     1      2     3     4    5    6   7   8   9   10  11  13  15
TYP_INT                                   

CF2        0.0   3.0   13.0  14.0  15.0  16.0  19.0  20.0  21.0  23.0  24.0  \
TYP_INT                                                                       
1        111577     3     2    26    42     3    55     9     2    90     7   
2         47863     1     0     3     4     0     2     4     1     9     0   
3         20777     0     0     2     0     0     2     3     0     6     0   
4           652     0     0     0     0     0     1     0     1     0     0   
5           143     0     0     0     0     0     0     0     0     0     0   
6           354     1     0     0     0     0     0     0     0     0     0   
7           247     0     0     0     0     0     0     0     0     0     0   
10          122     0     0     0     0     0     0     0     0     0     0   
98        22030     0     1     3     2     0     2     0     0     6     0   
99           82     0     0     0     0     0     0     0     0     0     0   

CF2      25.0  26.0  27.0  28.0  99.0  
TYP_INT    

RELJCT1_IM2     0.0   1.0    8.0  9.0
TYP_INT                              
1            119653  5809  17101   16
2             50971  1913   8512    1
3             21967   836   3585    3
4               658    42    125    0
5               128     3     49    0
6               412    33     53    0
7               335    32     40    0
10              124     1     19    0
11                5     1      0    0
98            21043   773   4746    3
99               71     6      5    3



In [12]:
Accident_Features = [
    ["ALCHL_IM", [], 2, 0],
    ["EVENT1_IM", [], 54, 0],
    ["HOUR_IM", [], 24, 0],
    ["INT_HWY", [9], 3, 25],
    ["LGTCON_IM", [], 7, 0],
    ["MANCOL_IM", [], 9, 0],
    ["MAXSEV_IM", [], 8, 0],
    ["MONTH", [], 12, 0],
    ["NO_INJ_IM", [], 18, 0],
    ["PEDS", [], 10, 0],
    ["PERMVIT", [], 26, 0],
    ["PERNOTMVIT", [], 10, 0],
    ["PSU", [], 60, 0],
    ["PVH_INVL", [], 11, 0],
    ["REGION", [], 4, 0],
    ["REL_ROAD", [98,99], 13, 190],
    ["RELJCT1_IM", [], 3, 0],
    ["RELJCT2_IM", [], 13, 0],
    ["SCH_BUS", [], 2, 0],
    ["TYP_INT", [98,99], 11, 26650],
    ["URBANICITY", [], 2, 0],
    ["VE_FORMS", [], 13, 0],
    ["VE_TOTAL", [], 13, 0],
    ["WEATHR_IM", [], 11, 0],
    ["WKDY_IM", [], 7, 0],
    ["WRK_ZONE", [], 5, 0],
    ["YEAR", [], 5, 0],
]

Accident_Features = sorted(Accident_Features, key=lambda x:x[0])
for feature in Accident_Features:
    print (feature)
print ()

Vehicle_Features = [
    # Features that CRSS imputed; use unimputed
    ['BODY_TYP', [98,99,49,79], 73, 18524],
    ['MOD_YEAR', [9998,9999], 83, 18524], 
    # Features with no unknown or missing values
    ['MODEL', [], 140, 0],
    # Features with unknown values, Not imputed by CRSS
    ['MAKE', [99], 70, 12901],
    ['MOD_YEAR', [9998,9999], 83, 18524],
    ['VALIGN', [8,9], 7, 31554],
    ['VNUM_LAN', [8,9], 10, 127387], # Count
    ['VPROFILE', [8,9], 9, 62776],
    ['VSPD_LIM', [98,99], 20, 62649],
    ['VTRAFCON', [97,99], 19, 30151],
    ['VTRAFWAY', [8,9], 9, 83513],
]

Vehicle_Features = sorted(Vehicle_Features, key=lambda x:x[0])
for feature in Vehicle_Features:
    print (feature)
print ()

Person_Features = [
    # Features that CRSS imputed; use unimputed
    ['AGE', [998,999], 188, 41087], # Person
    ['SEX', [8,9], 4, 26143],
    # Features with no unknown or missing values
    ['PER_TYP', [], 13, 0],
    # Features with unknown values, Not imputed by CRSS
    ['HOSPITAL', [8,9], 9, 13522],
]

Person_Features = sorted(Person_Features, key=lambda x:x[0])
for feature in Person_Features:
    print (feature)
print ()



['ALCHL_IM', [], 2, 0]
['EVENT1_IM', [], 54, 0]
['HOUR_IM', [], 24, 0]
['INT_HWY', [9], 3, 25]
['LGTCON_IM', [], 7, 0]
['MANCOL_IM', [], 9, 0]
['MAXSEV_IM', [], 8, 0]
['MONTH', [], 12, 0]
['NO_INJ_IM', [], 18, 0]
['PEDS', [], 10, 0]
['PERMVIT', [], 26, 0]
['PERNOTMVIT', [], 10, 0]
['PSU', [], 60, 0]
['PVH_INVL', [], 11, 0]
['REGION', [], 4, 0]
['RELJCT1_IM', [], 3, 0]
['RELJCT2_IM', [], 13, 0]
['REL_ROAD', [98, 99], 13, 190]
['SCH_BUS', [], 2, 0]
['TYP_INT', [98, 99], 11, 26650]
['URBANICITY', [], 2, 0]
['VE_FORMS', [], 13, 0]
['VE_TOTAL', [], 13, 0]
['WEATHR_IM', [], 11, 0]
['WKDY_IM', [], 7, 0]
['WRK_ZONE', [], 5, 0]
['YEAR', [], 5, 0]

['BODY_TYP', [98, 99, 49, 79], 73, 18524]
['MAKE', [99], 70, 12901]
['MODEL', [], 140, 0]
['MOD_YEAR', [9998, 9999], 83, 18524]
['MOD_YEAR', [9998, 9999], 83, 18524]
['VALIGN', [8, 9], 7, 31554]
['VNUM_LAN', [8, 9], 10, 127387]
['VPROFILE', [8, 9], 9, 62776]
['VSPD_LIM', [98, 99], 20, 62649]
['VTRAFCON', [97, 99], 19, 30151]
['VTRAFWAY', [8, 9], 9, 83

# def Erase Proportional Number of Samples from Each 

In [13]:
def Erase_Proportional(df, df_Original, df_Features):
    N = df_Original.shape[0]
    n = df.shape[0]
    print ("N = ", N, "n = ", n)
    print ()
    for F in df_Features:
        feature = F[0]
        nUnknown = int(F[3]*n/N + 0.5)
        print ()
        print (feature, n, N, df.shape[0], F[3], nUnknown)
        if nUnknown>0:
            A = random.sample(range(n), nUnknown)
#            A = sorted(A)
#            print (feature, A)
            for i in range (nUnknown):
                df.loc[A[i], feature] = ''
                
    return df
        

In [14]:
def Erase_Proportional_Test():
    df_Original = pd.DataFrame(np.random.randint(0,5,size=(20, 4)), columns=list('ABCD'))
    df = df_Original.copy(deep=True)
    df_Features = [
        ['A',0,0,0],
        ['B',0,0,0],
        ['C',0,0,0],
        ['D',0,0,0],
    ]
    for F in df_Features:
        feature = F[0]
        F[3] = len(df[df[feature]==0])
        
    for F in df_Features:
        feature = F[0]
        df.drop( df[ df[feature]==0].index, inplace=True)
    
    df = df.reset_index(drop=True)
        
    print (df_Original)
    print ()
    print (df_Features)
    print ()
    print (df)
    print ()
    df_New = Erase_Proportional(df, df_Original, df_Features)
    
    print ('df_New Again')
    print ()
    print (df_New)
    
        
Erase_Proportional_Test()

    A  B  C  D
0   2  1  1  4
1   4  3  1  0
2   4  2  3  1
3   4  1  1  1
4   0  0  4  1
5   0  4  0  0
6   0  0  2  3
7   1  3  2  3
8   1  3  2  3
9   1  4  2  4
10  3  2  3  3
11  0  0  1  2
12  2  4  2  0
13  1  1  0  1
14  4  2  3  1
15  1  3  4  4
16  1  2  3  2
17  3  2  1  4
18  2  4  1  3
19  3  1  1  4

[['A', 0, 0, 4], ['B', 0, 0, 3], ['C', 0, 0, 2], ['D', 0, 0, 3]]

    A  B  C  D
0   2  1  1  4
1   4  2  3  1
2   4  1  1  1
3   1  3  2  3
4   1  3  2  3
5   1  4  2  4
6   3  2  3  3
7   4  2  3  1
8   1  3  4  4
9   1  2  3  2
10  3  2  1  4
11  2  4  1  3
12  3  1  1  4

N =  20 n =  13


A 13 20 13 4 3

B 13 20 13 3 2

C 13 20 13 2 1

D 13 20 13 3 2
df_New Again

    A  B  C  D
0      1     4
1   4  2  3  1
2      1  1  1
3   1  3  2  3
4         2  3
5   1  4  2  4
6   3  2  3  3
7   4     3  1
8   1  3  4  4
9   1  2  3  2
10  3  2  1   
11  2  4  1   
12  3  1  1  4


# def Binning

In [15]:
def Build_Individual_Feature_with_Dict(df, feature, A):
    D = {}
    for B in A:
        for b in B[1]:
            D[b] = B[2]

    print (feature)
    print (D)
    print (df[feature].value_counts())
    print ('isna(): ', df[feature].isna().sum())

    df[feature].replace(D, inplace=True)
    
    print (df[feature].value_counts())
    print ()

    return df
    

In [16]:
def Test_Build_Individual_Feature_with_Dict():
    df = pd.DataFrame(np.random.randint(0,10,size=(20, 4)), columns=list('ABCD'))
    df['E'] = df['A']
    
    feature = 'A'
    A = [
        ['Low',[0,1,2], 10],
        ['Medium',[3,4,5,6], 20],
        ['High',[7,8,9], 30],
    ]
    
    print (df)
    print ()
    df = Build_Individual_Feature_with_Dict(df, feature, A)
    print (df)
    print ()
    
    return 0

Test_Build_Individual_Feature_with_Dict()


    A  B  C  D  E
0   6  1  1  5  6
1   7  8  3  4  7
2   9  8  5  3  9
3   7  9  7  6  7
4   6  9  7  2  6
5   5  9  7  3  5
6   0  5  0  1  0
7   5  8  4  8  5
8   4  4  4  5  4
9   0  8  3  7  0
10  4  8  4  3  4
11  8  0  0  8  8
12  0  8  9  6  0
13  3  0  9  9  3
14  4  9  7  0  4
15  2  1  6  8  2
16  8  8  6  5  8
17  5  0  6  1  5
18  8  6  8  4  8
19  2  1  2  5  2

A
{0: 10, 1: 10, 2: 10, 3: 20, 4: 20, 5: 20, 6: 20, 7: 30, 8: 30, 9: 30}
0    3
4    3
5    3
8    3
2    2
6    2
7    2
3    1
9    1
Name: A, dtype: int64
isna():  0
20    9
30    6
10    5
Name: A, dtype: int64

     A  B  C  D  E
0   20  1  1  5  6
1   30  8  3  4  7
2   30  8  5  3  9
3   30  9  7  6  7
4   20  9  7  2  6
5   20  9  7  3  5
6   10  5  0  1  0
7   20  8  4  8  5
8   20  4  4  5  4
9   10  8  3  7  0
10  20  8  4  3  4
11  30  0  0  8  8
12  10  8  9  6  0
13  20  0  9  9  3
14  20  9  7  0  4
15  10  1  6  8  2
16  30  8  6  5  8
17  20  0  6  1  5
18  30  6  8  4  8
19  10  1  2  5  2



0

In [29]:
def Bin_Accident_Dataset(df_Accident):
    print ('Bin_Accident_Dataset()')
    
    #feature = 'DAY_WEEK'
    feature = 'WKDY_IM'
    A = [
        ['Weekend', [1,7], 0],
        ['Weekday', [2,3,4,5,6], 1],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)
    
    feature = 'EVENT1_IM'
    A = [
        ["I", [74, 10, 32, 1, 5, 42, 21, 35, 58, 19, 93, 6, 49, 34, 46, 52, 30, 39, 33, 20], 8],
        ["H", [8], 7],
        ["G", [41, 44, 23, 24, 26, 17, 31, 7, 15, 25, 38, 53], 6],
        ["F", [9], 5],
        ["E", [59, 43, 45, 3, 91, 40, 57], 4],
        ["D", [55, 48, 18], 3],
        ["C", [12], 2],
        ["B", [14], 1],
        ["A", [16, 51, 50, 73, 11, 2, 72, 54], 0],  
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)
        

#    feature = 'HOUR'
    feature = 'HOUR_IM'
    A = [
        ['Early_Morn', [5,6], 0],
        ['Morning', [7,8,9,10], 1],
        ['Mid_Day', [11,12,13,14], 2],
        ['Rush_Hour', [15,16,17], 3],
        ['Early_Eve', [18,19], 4],
        ['Evening', [20,21,22], 5],
        ['Late_Nght',[23,0,1,2,3,4], 6],
             ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'INT_HWY'
    A = [
        ['No', [0], 0],
        ['Yes', [1], 1],
#        ['Missing', [9], 999],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

#    feature = 'LGT_COND'
    feature = 'LGTCON_IM'
    A =  [
        ['Dark', [2], 0],
        ['Dawn_Lighted', [3,4,6], 1],
        ['Dusk', [5], 2],
        ['Daylight', [1,7], 3],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'MONTH'
    A = [
        ['Winter', [1,2,3,12], 0],
        ['Spring_Fall', [4,5,10,11], 1],
        ['Summer', [6,7,8,9], 2],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    
    feature = 'NO_INJ_IM'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [0,1,2]]
#    print (B)
    A = [
        ['0', [0], 0],
        ['1', [1], 1],
        ['2', [2], 2],
        ['Multiple', B, 3]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)
    
    # PEDS is derived 
    # "This data element is the number of Person Forms (Not a Motor Vehicle Occupant) 
    # that are applicable to this case (i.e., non-occupants)."
    # I've changed it from a count variable to a binary variable.  
    feature = 'PEDS'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [0]]
#    print (B)
    A = [
        ['No', [0], 0],
        ['Yes', B, 1],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    # Derived from PERSON file
    # "[Number of] Persons in Motor Vehicle in Transport"
    feature = 'PERMVIT'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [1,2]]
#    print (B)
    A = [
        ['1', [1], 0],
        ['2', [2], 1],
        ['Multiple', B, 2]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'PERNOTMVIT'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [1,2]]
#    print (B)
    A = [
        ['1', [1], 0],
        ['2', [2], 1],
        ['Multiple', B, 2]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'PVH_INVL'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [1,2]]
#    print (B)
    A = [
        ['1', [1], 0],
        ['2', [2], 1],
        ['Multiple', B, 2]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'REL_ROAD'
    A =  [
        ['Not_on_Road', [2,3,4,5,6,8,10,12], 0],
        ['On_Road', [1,11], 1],
        ['Parking_Area', [7], 2],
#        ['Missing/Unknown', [98,99], 999]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'REGION'
    A = [
        ['Northeast', [1], 0],
        ['Midwest', [2], 1],
        ['South', [3], 2],
        ['West', [4], 3]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

#    feature = 'RELJCT1'
    feature = 'RELJCT1_IM'
    A = [
        ['No', [0], 0],
        ['Yes', [1], 1],
#        ['Missing', [8,9], 999]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

#    feature = 'RELJCT2'
    feature = 'RELJCT2_IM'
    A = [
        ['A', [2,5,6,19], 0],
        ['B', [1,7,16], 1],
        ['C', [4,8,18], 2],
        ['D', [3,17,20], 3],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'SCH_BUS'
    A = [
        ['No', [0], 0],
        ['Yes', [1], 1]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'TYP_INT'
    A = [
        ['Not an Intersection', [1], 0],
        ['Intersection', [2,3,4,7,10,11], 1],
        ['Roundabout', [5,6], 2],
#        ['Unknown', [98,99], 999]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'URBANICITY'
    A = [
        ['Urban', [1], 0],
        ['Rural', [2], 1]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    # Derived from VEHICLE file
    feature = 'VE_FORMS'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [1,2,3]]
#    print (B)
    A = [
        ['1', [1], 0],
        ['2', [2], 1],
        ['3', [3], 2],
        ['Multiple', B, 3]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'VE_TOTAL'
    B = [x for x in list(df_Accident[feature].unique()) if x not in [1,2,3]]
#    print (B)
    A = [
        ['1', [1], 0],
        ['2', [2], 1],
        ['3', [3], 2],
        ['Multiple', B, 3]
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)


#    feature = 'WEATHER'
    feature = 'WEATHR_IM'
    A = [
        ['A', [3,5], 0],
        ['B', [1], 1],
        ['C', [2], 2],
        ['D', [10], 3],
        ['E', [4,6,7,8,11,12], 4],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)

    feature = 'WRK_ZONE'
    A = [
        ['0', [0], 0],
        ['1', [1,2,3,4], 1],
    ]
    df_Accident = Build_Individual_Feature_with_Dict(df_Accident, feature, A)
    
    print ()
    return df_Accident

In [30]:
# Test Bin_Accident_Dataset
A = pd.DataFrame([])
for f in Accident_Features:
    feature = f[0]
    A[feature] = df_Accident[feature]
A = Bin_Accident_Dataset(A)
for feature in A:
    n = len(A[feature].unique())
    print (feature, n)

Bin_Accident_Dataset()
WKDY_IM
{1: 0, 7: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1}
1    197152
0     61925
Name: WKDY_IM, dtype: int64
isna():  0
0    259077
Name: WKDY_IM, dtype: int64

EVENT1_IM
{74: 8, 10: 8, 32: 8, 1: 8, 5: 8, 42: 8, 21: 8, 35: 8, 58: 8, 19: 8, 93: 8, 6: 8, 49: 8, 34: 8, 46: 8, 52: 8, 30: 8, 39: 8, 33: 8, 20: 8, 8: 7, 41: 6, 44: 6, 23: 6, 24: 6, 26: 6, 17: 6, 31: 6, 7: 6, 15: 6, 25: 6, 38: 6, 53: 6, 9: 5, 59: 4, 43: 4, 45: 4, 3: 4, 91: 4, 40: 4, 57: 4, 55: 3, 48: 3, 18: 3, 12: 2, 14: 1, 16: 0, 51: 0, 50: 0, 73: 0, 11: 0, 2: 0, 72: 0, 54: 0}
12    171111
8      12094
14      9532
11      8986
9       8538
1       6566
34      5080
42      4832
30      4099
33      3737
24      3111
35      2219
59      2005
25      1920
38      1758
18      1529
53      1483
43      1185
32       925
15       800
31       789
54       729
52       573
57       538
39       534
5        462
19       440
40       374
23       273
41       234
46       223
17       186
26       170
44       170

# Plan

- In the CRSS dataset, we want to do two things.  
    - Impute unknown values
    - Bin into fewer categories
- Does the order of operations matter?
- General Strategy
    - Pull the features I want to use in the Accident data file
        - If the feature has been imputed by CRSS, pull the unimputed version
    - For each feature, count the number of samples with unknown values and record the proportion, p(feature)
    - Delete any records with unknown values in any features.  Call this dataframe df_A.
    - Create a deep copy of df_A with binned values to be ground truth.  Call this dataframe df_B.
    - Repeat this part twice with different random seeds for the deletion and imputation
        - For each feature, delete the value for p(feature) of the records. Call this dataframe df_C.
        - Bin Then Impute
            - Bin the values in df_C; call it df_D.
            - Impute blank values in df_D; call it df_E
        - Impute Then Bin
            - Impute blank values in df_C; call it df_F.
            - Bin the values in df_F; call it df_G

## Create df_A:  df_Accident with Unknown Values Removed

In [31]:
df_A = pd.DataFrame([])
for F in Accident_Features:
    feature = F[0]
    df_A[feature] = df_Accident[feature]

for F in Accident_Features:
    feature = F[0]
    Unknown = F[1]
    df_A.drop( df_A[ df_A[feature].isin(Unknown)].index, inplace=True)
    print (feature, len(df_A))
print ()

df_A = df_A.reset_index(drop=True)

for feature in df_A:
    print (feature, len(df_A[feature].unique()))

ALCHL_IM 259077
EVENT1_IM 259077
HOUR_IM 259077
INT_HWY 259077
LGTCON_IM 259077
MANCOL_IM 259077
MAXSEV_IM 259077
MONTH 259077
NO_INJ_IM 259077
PEDS 259077
PERMVIT 259077
PERNOTMVIT 259077
PSU 259077
PVH_INVL 259077
REGION 259077
RELJCT1_IM 259077
RELJCT2_IM 259077
REL_ROAD 259077
SCH_BUS 259077
TYP_INT 259077
URBANICITY 259077
VE_FORMS 259077
VE_TOTAL 259077
WEATHR_IM 259077
WKDY_IM 259077
WRK_ZONE 259077
YEAR 259077

ALCHL_IM 2
EVENT1_IM 54
HOUR_IM 7
INT_HWY 3
LGTCON_IM 4
MANCOL_IM 9
MAXSEV_IM 8
MONTH 3
NO_INJ_IM 18
PEDS 2
PERMVIT 3
PERNOTMVIT 3
PSU 60
PVH_INVL 3
REGION 4
RELJCT1_IM 3
RELJCT2_IM 4
REL_ROAD 4
SCH_BUS 2
TYP_INT 4
URBANICITY 2
VE_FORMS 4
VE_TOTAL 4
WEATHR_IM 5
WKDY_IM 2
WRK_ZONE 2
YEAR 5


## Create df_B with binned values to be ground truth

In [32]:
df_B = df_A.copy(deep=True)
df_B = Bin_Accident_Dataset(df_B)

Bin_Accident_Dataset()
WKDY_IM
{1: 0, 7: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1}
1    197152
0     61925
Name: WKDY_IM, dtype: int64
isna():  0
0    259077
Name: WKDY_IM, dtype: int64

EVENT1_IM
{74: 8, 10: 8, 32: 8, 1: 8, 5: 8, 42: 8, 21: 8, 35: 8, 58: 8, 19: 8, 93: 8, 6: 8, 49: 8, 34: 8, 46: 8, 52: 8, 30: 8, 39: 8, 33: 8, 20: 8, 8: 7, 41: 6, 44: 6, 23: 6, 24: 6, 26: 6, 17: 6, 31: 6, 7: 6, 15: 6, 25: 6, 38: 6, 53: 6, 9: 5, 59: 4, 43: 4, 45: 4, 3: 4, 91: 4, 40: 4, 57: 4, 55: 3, 48: 3, 18: 3, 12: 2, 14: 1, 16: 0, 51: 0, 50: 0, 73: 0, 11: 0, 2: 0, 72: 0, 54: 0}
12    171111
8      12094
14      9532
11      8986
9       8538
1       6566
34      5080
42      4832
30      4099
33      3737
24      3111
35      2219
59      2005
25      1920
38      1758
18      1529
53      1483
43      1185
32       925
15       800
31       789
54       729
52       573
57       538
39       534
5        462
19       440
40       374
23       273
41       234
46       223
17       186
26       170
44       170

## Create df_C from df_A with Erased Samples

In [33]:
df_C = df_A.copy(deep=True)
df_C = Erase_Proportional(df_C, df_Accident, Accident_Features)
df_C2 = df_A.copy(deep=True)
df_C2 = Erase_Proportional(df_C2, df_Accident, Accident_Features)
print (df_C.head(20))

N =  259077 n =  259077


ALCHL_IM 259077 259077 259077 0 0

EVENT1_IM 259077 259077 259077 0 0

HOUR_IM 259077 259077 259077 0 0

INT_HWY 259077 259077 259077 25 25

LGTCON_IM 259077 259077 259077 0 0

MANCOL_IM 259077 259077 259077 0 0

MAXSEV_IM 259077 259077 259077 0 0

MONTH 259077 259077 259077 0 0

NO_INJ_IM 259077 259077 259077 0 0

PEDS 259077 259077 259077 0 0

PERMVIT 259077 259077 259077 0 0

PERNOTMVIT 259077 259077 259077 0 0

PSU 259077 259077 259077 0 0

PVH_INVL 259077 259077 259077 0 0

REGION 259077 259077 259077 0 0

RELJCT1_IM 259077 259077 259077 0 0

RELJCT2_IM 259077 259077 259077 0 0

REL_ROAD 259077 259077 259077 190 190

SCH_BUS 259077 259077 259077 0 0

TYP_INT 259077 259077 259077 26650 26650

URBANICITY 259077 259077 259077 0 0

VE_FORMS 259077 259077 259077 0 0

VE_TOTAL 259077 259077 259077 0 0

WEATHR_IM 259077 259077 259077 0 0

WKDY_IM 259077 259077 259077 0 0

WRK_ZONE 259077 259077 259077 0 0

YEAR 259077 259077 259077 0 0
N =  259077 n =  259077




# Bin Before Imputing
- Bin the values in df_C; call it df_D.
- Impute blank values in df_D; call it df_E
- For each feature, for the samples that were blank, make a crosstab between df_B and df_E


## df_D is df_C binned

In [34]:
df_D = df_C.copy(deep=True)
df_D = Bin_Accident_Dataset(df_D)
#df_D.to_csv('../../Big_Files/OoO_10_19_22_Accident_df_D.txt', sep='\t', index=False)
df_D.to_csv('../../Big_Files/OoO_11_01_22_Accident_df_D.txt', sep='\t', index=False)

df_D2 = df_C2.copy(deep=True)
df_D2 = Bin_Accident_Dataset(df_D2)
#df_D2.to_csv('../../Big_Files/OoO_10_19_22_Accident_df_D2.txt', sep='\t', index=False)
df_D2.to_csv('../../Big_Files/OoO_11_01_22_Accident_df_D2.txt', sep='\t', index=False)


Bin_Accident_Dataset()
WKDY_IM
{1: 0, 7: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1}
1    197152
0     61925
Name: WKDY_IM, dtype: int64
isna():  0
0    259077
Name: WKDY_IM, dtype: int64

EVENT1_IM
{74: 8, 10: 8, 32: 8, 1: 8, 5: 8, 42: 8, 21: 8, 35: 8, 58: 8, 19: 8, 93: 8, 6: 8, 49: 8, 34: 8, 46: 8, 52: 8, 30: 8, 39: 8, 33: 8, 20: 8, 8: 7, 41: 6, 44: 6, 23: 6, 24: 6, 26: 6, 17: 6, 31: 6, 7: 6, 15: 6, 25: 6, 38: 6, 53: 6, 9: 5, 59: 4, 43: 4, 45: 4, 3: 4, 91: 4, 40: 4, 57: 4, 55: 3, 48: 3, 18: 3, 12: 2, 14: 1, 16: 0, 51: 0, 50: 0, 73: 0, 11: 0, 2: 0, 72: 0, 54: 0}
12    171111
8      12094
14      9532
11      8986
9       8538
1       6566
34      5080
42      4832
30      4099
33      3737
24      3111
35      2219
59      2005
25      1920
38      1758
18      1529
53      1483
43      1185
32       925
15       800
31       789
54       729
52       573
57       538
39       534
5        462
19       440
40       374
23       273
41       234
46       223
17       186
26       170
44       170

1    134992
0     80913
2     43172
Name: REGION, dtype: int64

RELJCT1_IM
{0: 0, 1: 1, 8: 999, 9: 999}
0.0    196668
1.0      8000
Name: RELJCT1_IM, dtype: int64
isna():  54409
0.0    196668
1.0      8000
Name: RELJCT1_IM, dtype: int64

RELJCT2_IM
{2: 0, 5: 0, 6: 0, 19: 0, 1: 1, 7: 1, 16: 1, 4: 2, 8: 2, 18: 2, 3: 3, 17: 3, 20: 3}
1    111598
0     62044
3     62008
2     23427
Name: RELJCT2_IM, dtype: int64
isna():  0
1    111598
0     85471
3     62008
Name: RELJCT2_IM, dtype: int64

SCH_BUS
{0: 0, 1: 1}
0    257706
1      1371
Name: SCH_BUS, dtype: int64
isna():  0
0    257706
1      1371
Name: SCH_BUS, dtype: int64

TYP_INT
{1: 0, 2: 1, 3: 1, 4: 1, 7: 1, 10: 1, 11: 1, 5: 2, 6: 2, 98: 999, 99: 999}
0      127953
1       79947
        26650
999     23921
2         606
Name: TYP_INT, dtype: int64
isna():  0
0      207900
        26650
999     23921
1         606
Name: TYP_INT, dtype: int64

URBANICITY
{1: 0, 2: 1}
0    198588
1     60489
Name: URBANICITY, dtype: int64
isna():  0
0    

## Do the Imputation in IVEware
- df_D to df_E
- df_D2 to df_E2

## df_E is df_D (binned) with Missing Values Imputed

In [35]:
#df_E = pd.read_csv('../../Big_Files/OoO_10_19_22_Accident_df_E.csv')
#df_E2 = pd.read_csv('../../Big_Files/OoO_10_19_22_Accident_df_E2.csv')
df_E = pd.read_csv('../../Big_Files/OoO_11_01_22_Accident_df_E.csv')
df_E2 = pd.read_csv('../../Big_Files/OoO_11_01_22_Accident_df_E2.csv')

# Impute Before Binning
- Impute blank values in df_C; call it df_F.
- Bin the values in df_F; call it df_G
- For each feature, for the samples that were blank, make a crosstab between df_B and df_G


In [36]:
#df_C.to_csv('../../Big_Files/OoO_10_19_22_Accident_df_C.txt', sep='\t', index=False)
#df_C2.to_csv('../../Big_Files/OoO_10_19_22_Accident_df_C2.txt', sep='\t', index=False)
df_C.to_csv('../../Big_Files/OoO_11_01_22_Accident_df_C.txt', sep='\t', index=False)
df_C2.to_csv('../../Big_Files/OoO_11_01_22_Accident_df_C2.txt', sep='\t', index=False)

## Do the Imputation in IVEware
- df_C to df_F
- df_C2 to df_F2

## df_F is df_C (unbinned) with Missing Values Imputed
## df_G is df_F binned

In [37]:
#df_F = pd.read_csv('../../Big_Files/OoO_10_19_22_Accident_df_F.csv')
#df_F2 = pd.read_csv('../../Big_Files/OoO_10_19_22_Accident_df_F2.csv')
df_F = pd.read_csv('../../Big_Files/OoO_11_01_22_Accident_df_F.csv')
df_F2 = pd.read_csv('../../Big_Files/OoO_11_01_22_Accident_df_F2.csv')
df_G = Bin_Accident_Dataset(df_F)
df_G2 = Bin_Accident_Dataset(df_F2)

Bin_Accident_Dataset()
WKDY_IM
{1: 0, 7: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1}
1    197152
0     61925
Name: WKDY_IM, dtype: int64
isna():  0
0    259077
Name: WKDY_IM, dtype: int64

EVENT1_IM
{74: 8, 10: 8, 32: 8, 1: 8, 5: 8, 42: 8, 21: 8, 35: 8, 58: 8, 19: 8, 93: 8, 6: 8, 49: 8, 34: 8, 46: 8, 52: 8, 30: 8, 39: 8, 33: 8, 20: 8, 8: 7, 41: 6, 44: 6, 23: 6, 24: 6, 26: 6, 17: 6, 31: 6, 7: 6, 15: 6, 25: 6, 38: 6, 53: 6, 9: 5, 59: 4, 43: 4, 45: 4, 3: 4, 91: 4, 40: 4, 57: 4, 55: 3, 48: 3, 18: 3, 12: 2, 14: 1, 16: 0, 51: 0, 50: 0, 73: 0, 11: 0, 2: 0, 72: 0, 54: 0}
12    171111
8      12094
14      9532
11      8986
9       8538
1       6566
34      5080
42      4832
30      4099
33      3737
24      3111
35      2219
59      2005
25      1920
38      1758
18      1529
53      1483
43      1185
32       925
15       800
31       789
54       729
52       573
57       538
39       534
5        462
19       440
40       374
23       273
41       234
46       223
17       186
26       170
44       170

0    123107
1     94919
2     27377
3     13674
Name: NO_INJ_IM, dtype: int64

PEDS
{0: 0, 1: 1}
0    236835
1     22242
Name: PEDS, dtype: int64
isna():  0
0    236835
1     22242
Name: PEDS, dtype: int64

PERMVIT
{1: 0, 2: 1, 0: 2}
1    100311
2     89183
0     69583
Name: PERMVIT, dtype: int64
isna():  0
0    100311
1     89183
2     69583
Name: PERMVIT, dtype: int64

PERNOTMVIT
{1: 0, 2: 1, 0: 2}
2    235594
0     22624
1       859
Name: PERNOTMVIT, dtype: int64
isna():  0
1    235594
2     22624
0       859
Name: PERNOTMVIT, dtype: int64

PVH_INVL
{1: 0, 2: 1, 0: 2}
2    248068
0      9809
1      1200
Name: PVH_INVL, dtype: int64
isna():  0
1    248068
2      9809
0      1200
Name: PVH_INVL, dtype: int64

REL_ROAD
{2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 8: 0, 10: 0, 12: 0, 1: 1, 11: 1, 7: 2, 98: 999, 99: 999}
1      207306
0       43998
2        7583
999       190
Name: REL_ROAD, dtype: int64
isna():  0
1      207306
0       51581
999       190
Name: REL_ROAD, dtype: int64

REGION
{1: 0, 2

## df_H is df_D with missing values filled with the mode

In [38]:
df_H = pd.DataFrame([])
df_H2 = pd.DataFrame([])
for feature in df_D:
    mode = df_D[feature].mode(dropna=True)
    m = mode.tolist()[0]
    df_H[feature] = df_D[feature].replace({'':m})
    mode = df_D2[feature].mode(dropna=True)
    m = mode.tolist()[0]
    df_H2[feature] = df_D2[feature].replace({'':m})
    print (feature)
    print ()
    print (df_D[feature].value_counts())
    print ()
    print (df_H[feature].value_counts())
    print ()


ALCHL_IM

2    243506
1     15571
Name: ALCHL_IM, dtype: int64

2    243506
1     15571
Name: ALCHL_IM, dtype: int64

EVENT1_IM

2    171111
8     30360
7     12094
6     11024
0     10428
1      9532
5      8538
4      4377
3      1613
Name: EVENT1_IM, dtype: int64

2    171111
8     30360
7     12094
6     11024
0     10428
1      9532
5      8538
4      4377
3      1613
Name: EVENT1_IM, dtype: int64

HOUR_IM

6    211198
0     47879
Name: HOUR_IM, dtype: int64

6    211198
0     47879
Name: HOUR_IM, dtype: int64

INT_HWY

0      235097
1       23930
999        25
           25
Name: INT_HWY, dtype: int64

0      235122
1       23930
999        25
Name: INT_HWY, dtype: int64

LGTCON_IM

1    178210
3     47702
0     33165
Name: LGTCON_IM, dtype: int64

1    178210
3     47702
0     33165
Name: LGTCON_IM, dtype: int64

MANCOL_IM

0     87223
1     75394
6     58039
7     23308
2      8251
8      3534
9      1800
11     1401
10      127
Name: MANCOL_IM, dtype: int64

0     87223
1     

# Analysis

## Plan
- We have these dataframes:
    - df_B is ground truth for binning
    - df_C is our record of which values we deleted
    - df_E is Bin Before Imputing
    - df_G is Impute before Binning
- For each feature:
    - Make df_BA to be samples in df_B whose values in df_C are blank
    - Make df_EA to be samples in df_E whose values in df_C are blank
    - Make df_GA to be samples in df_G whose values in df_C are blank
    - Make crosstabs between (df_BA and df_EA) and (df_BA and df_GA) and (df_EA and df_GA)
    - Count correct imputation; divide by number of samples to give proportion correct
- After the second round of imputation:
    - Make df_BB to be samples in df_B whose values in df_C are blank
    - Make df_EB to be samples in df_E whose values in df_C are blank
    - Make df_GB to be samples in df_G whose values in df_C are blank
    - Make crosstabs between (df_BB and df_EB) and (df_BB and df_GB) and (df_EA and df_EB) and (df_GA and df_GB)
    - Count correct imputation; divide by number of samples to give proportion correct


In [39]:
def Crosstabs(df_C1, df_C2, df_1, df_2, text_1, text_2, feature):
    df_1A = df_1[feature]
    df_2A = df_2[feature]
    df_1A = df_1A[df_C1[feature] == '']
    df_2A = df_2A[df_C2[feature] == '']
    CT = pd.crosstab(df_1A, df_2A, rownames = [text_1], colnames = [text_2])
    A = CT.values.tolist()
    s = 0
    S = 0
    for i in range (len(A)):
        for j in range (len(A[0])):
            S += A[i][j]
            if i==j:
                s += A[i][j]
    print (feature, text_1, text_2)
    print ()
    print (CT)
    print ()
    print (S, s, round(s/S*100,2), '%')
    print ()
    print (CT.to_latex())
#    print ()
    print ()

In [40]:
#feature = 'WEATHER'
#feature = 'HOUR'
#feature = 'INT_HWY'
#feature = 'REL_ROAD'
feature = 'TYP_INT'
print (df_Accident.shape[0])
print (df_Accident[feature].value_counts())
print ()
print (df_Accident[feature].value_counts(normalize=True))
print ()
print (df_A[feature].value_counts())
print ()
print (df_A[feature].value_counts(normalize=True))
print ()
print (df_B[feature].value_counts())
print ()
print (df_B[feature].value_counts(normalize=True))
print ()
Crosstabs(df_C, df_C, df_B, df_B, 'Ground_Truth_1', 'Ground_Truth_1', feature)
Crosstabs(df_C2, df_C2, df_B, df_B, 'Ground_Truth_2', 'Ground_Truth_2', feature)
Crosstabs(df_C, df_C, df_B, df_E, 'Ground_Truth_1', 'Bin_Impute_1', feature)
Crosstabs(df_C2, df_C2, df_B, df_E2, 'Ground_Truth_2', 'Bin_Impute_2', feature)
Crosstabs(df_C, df_C, df_B, df_G, 'Ground_Truth_1', 'Impute_Bin_1', feature)
Crosstabs(df_C2, df_C2, df_B, df_G2, 'Ground_Truth_2', 'Impute_Bin_2', feature)
Crosstabs(df_C, df_C, df_E, df_G, 'Bin_Impute_1', 'Impute_Bin_1', feature)
Crosstabs(df_C2, df_C2, df_E2, df_G2, 'Bin_Impute_2', 'Impute_Bin_2', feature)
#Crosstabs(df_C, df_C2, df_E, df_E2, 'Bin_Impute_1', 'Bin_Impute_2', feature)
Crosstabs(df_C, df_C, df_B, df_H, 'Ground_Truth_1', 'Impute_to_Mode_1', feature)
Crosstabs(df_C2, df_C2, df_B, df_H2, 'Ground_Truth_2', 'Impute_to_Mode_2', feature)



259077
0      142579
1       89170
999     26650
2         678
Name: TYP_INT, dtype: int64

0      0.550334
1      0.344183
999    0.102865
2      0.002617
Name: TYP_INT, dtype: float64

0      142579
1       89170
999     26650
2         678
Name: TYP_INT, dtype: int64

0      0.550334
1      0.344183
999    0.102865
2      0.002617
Name: TYP_INT, dtype: float64

0      231749
999     26650
1         678
Name: TYP_INT, dtype: int64

0      0.894518
999    0.102865
1      0.002617
Name: TYP_INT, dtype: float64

TYP_INT Ground_Truth_1 Ground_Truth_1

Ground_Truth_1    0    1     999
Ground_Truth_1                  
0               23806    0     0
1                   0   59     0
999                 0    0  2785

26650 26650 100.0 %

\begin{tabular}{lrrr}
\toprule
Ground\_Truth\_1 &    0   &  1   &   999 \\
Ground\_Truth\_1 &        &      &       \\
\midrule
0              &  23806 &    0 &     0 \\
1              &      0 &   59 &     0 \\
999            &      0 &    0 &  2785 \\
\bo